# oct-image-enhancement-optimisation
### CITS4404 Assignment 2 @ The University of Western Australia

Alian Haidar (22900426)\
Conan Dewitt (22877792)\
Dennis Gunadi (22374535)\
Lili Liu (23212326)

This project attempt presents the optimisation of a Camo Worm based image enhancement algorithm, specifically in the context of optical coherence tomography.

Import the necessary modules:

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import imageio.v3 as iio

Include the custom [worm.py](worm.py) module for visualisation:

In [ ]:
import worm as wrm

Define global variables:

In [ ]:
rng = np.random.default_rng()
Path = mpath.Path

mpl.rcParams['figure.dpi']= 72  # size of image

POPULATION_SIZE = 40
IMAGE_DIR = 'images'
IMAGE_NAME = 'original'
MASK = [320, 560, 160, 880]  # y_min, y_max, x_min, x_max

Read, resize and display the input image:

In [ ]:
def crop(image, mask):
    h, w = np.shape(image)
    return image[max(mask[0],0):min(mask[1],h), max(mask[2],0):min(mask[3],w)]

def prep_image(imdir, imname, mask):
    image = np.flipud(crop(iio.imread(imdir+'/'+imname+".png"), mask))
    print("Image name:    ", imname)
    print("Image size:    ", *np.shape(image))
    print("Intensity max: ", np.max(image))
    print("Intensity min: ", np.min(image))
    print("Intensity mean:", round(np.mean(image), 1))
    print("Intensity std: ", round(np.std(image), 1))
    plt.imshow(image, vmin=0, vmax=255, cmap='gray', origin='lower')  # use vmin and vmax to stop imshow from scaling
    plt.show()
    return image

image = prep_image(IMAGE_DIR, IMAGE_NAME, MASK)

Present the Cost Function:

In [ ]:
def calculate_cost(worm, image, other_worms):
    # Extract target colour
    target_colour = image[int(worm.y), int(worm.x)] / 255.0
    
    # Generate Bézier curve points
    x_points, y_points = worm.bezier_points()
    
    # Ensure points are within image boundaries
    x_points = np.clip(x_points, 0, image.shape[1]-1)
    y_points = np.clip(y_points, 0, image.shape[0]-1)
    
    # Calculate environment cost
    worm_pixels = image[y_points, x_points]
    environment_cost = np.mean(np.abs(worm_pixels - target_colour))
    
    # Calculate group knowledge cost
    position_cost = 0
    for other_worm in other_worms:
        if other_worm != worm:
            distance = np.sqrt((worm.x - other_worm.x)**2 + (worm.y - other_worm.y)**2)
            position_cost += distance

    # Calculate internal knowledge costs
    size_cost = worm.r
    
    p0 = (worm.x, worm.y)
    p1 = (worm.x + worm.dr * np.cos(worm.theta + worm.dgamma), 
          worm.y + worm.dr * np.sin(worm.theta + worm.dgamma))
    p2 = (worm.x + worm.r * np.cos(worm.theta), 
          worm.y + worm.r * np.sin(worm.theta))
    vector1 = np.array(p1) - np.array(p0)
    vector2 = np.array(p2) - np.array(p1)
    angle = np.arccos(np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))
    curvature_cost = angle
    
    color_cost = np.mean(np.abs(worm.colour - target_colour))

    # Combine costs
    total_cost = size_cost + curvature_cost + color_cost + position_cost + environment_cost
    return total_cost

Define the Worm Population, the *Genetic Function*:

In [ ]:
class Worm_Population:
    # Initialize the Worm_Population object with a given size and image shape
    def __init__(self, size, image_shape):
        self.worms = [self.create_random_worm(image_shape) for _ in range(size)]
        self.image_shape = image_shape

    # Create a random worm with random attributes based on the given image shape
    def create_random_worm(self, image_shape):
        # Randomly generate attributes for the worm
        x = np.random.randint(0, image_shape[1])
        y = np.random.randint(0, image_shape[0])
        r = np.random.randint(10, 100)
        theta = np.random.rand() * 2 * np.pi
        deviation_r = np.random.randint(-50, 50)
        deviation_gamma = np.random.rand() * 2 * np.pi
        width = np.random.randint(1, 10)
        colour = np.random.rand()
        return wrm.Camo_Worm(x, y, r, theta, deviation_r, deviation_gamma, width, colour)

    # Evaluate the fitness of each worm in the population based on a given image and other worms
    def evaluate_fitness(self, image, other_worms):
        for worm in self.worms:
            worm.fitness = 1 / (1 + calculate_cost(worm, image, other_worms))

    # Select the fittest worms to survive and reproduce
    def select(self):
        self.worms.sort(key=lambda x: x.fitness, reverse=True)
        self.worms = self.worms[:len(self.worms)//2] * 2  # keep top 50% and duplicate to maintain population size

    # Perform crossover to generate a new generation of worms
    def crossover(self):
        new_generation = []
        while len(new_generation) < len(self.worms) * 2:
            if len(self.worms) >= 2:
                parent1, parent2 = np.random.choice(self.worms, 2, replace=False)  # Randomly select parents
                child1, child2 = self.mate(parent1, parent2)  # Generate children through crossover
                new_generation.extend([child1, child2])  # Add children to new generation
            else:
                break
        self.worms = new_generation[:len(self.worms)]  # Update population with new generation

    # Generate children by averaging attributes of two parent worms
    def mate(self, parent1, parent2):
        child1 = wrm.Camo_Worm(  # Create child 1
            (parent1.x + parent2.x) // 2,
            (parent1.y + parent2.y) // 2,
            (parent1.r + parent2.r) // 2,
            (parent1.theta + parent2.theta) / 2,
            (parent1.dr + parent2.dr) // 2,
            (parent1.dgamma + parent2.dgamma) / 2,
            (parent1.width + parent2.width) // 2,
            (parent1.colour + parent2.colour) / 2
        )
        child2 = wrm.Camo_Worm(  # Create child 2
            (parent1.x + parent2.x) // 2,
            (parent1.y + parent2.y) // 2,
            (parent1.r + parent2.r) // 2,
            (parent1.theta + parent2.theta) / 2,
            (parent1.dr + parent2.dr) // 2,
            (parent1.dgamma + parent2.dgamma) / 2,
            (parent1.width + parent2.width) // 2,
            (parent1.colour + parent2.colour) / 2
        )
        return child1, child2  # Return the generated children

Create the Camo Worm population about the input image, and visualise each generation:

In [ ]:
# Run the genetic algorithm
def run_algorithm(image, generations=40):
    population = Worm_Population(POPULATION_SIZE, image.shape)
    for i in range(generations):
        population.evaluate_fitness(image, population.worms)
        population.select()
        population.crossover()
        
        # Update attributes of worms in the population
        for worm in population.worms:
            worm.grow()
            worm.move(image.shape)
            worm.adapt_color(image)
            worm.adapt_curvature()
        
        # Display generation details
        print(f"Generation {i+1}: {len(population.worms)} worms")
        
        # Calculate and display the total cost of the population
        total_cost = 0
        for worm in population.worms:
            cost = calculate_cost(worm, image, population.worms)
            total_cost += cost
        average_cost = total_cost / len(population.worms)
        print(f"Average Cost: {average_cost}")

        # Display generation
        visualize_population(population.worms, image)
    
    return population

# Visualize the population of worms
def visualize_population(worms, image):
    drawing = wrm.Drawing(image)
    drawing.add_worms(worms)
    drawing.show()

# Example usage
optimized_population = run_algorithm(image)
drawing = wrm.Drawing(image)
drawing.add_worms(optimized_population.worms)
drawing.show()